In [ ]:
import pandas as pd
import numpy as np

In [ ]:
root_dir ="/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/ExperimentResults/"
extra_path = ("QuadSinHF_NewLongerDriftEvalExp_MSEs_1024NPaths").replace(
            ".", "")

In [ ]:
true_paths = np.load(root_dir+extra_path+"_true_paths.npy", allow_pickle=True)[:, 1:, 0]
score_paths = np.load(root_dir+extra_path+"_score_paths.npy", allow_pickle=True)[:, 1:, 0]
nad_paths = np.load(root_dir+extra_path+"_nad_paths.npy", allow_pickle=True)[:, 1:, 0]
ridge_paths = np.load(root_dir+extra_path+"_ridge_paths.npy", allow_pickle=True)[:, 1:, 0]
hermite_paths = np.load(root_dir+extra_path+"_hermite_paths.npy", allow_pickle=True)[:, 1:, 0]

#true_drifts= np.load(root_dir+extra_path+"_true_drifts.npy", allow_pickle=True)[:, 1:, 0]
#score_drifts = np.load(root_dir+extra_path+"_score_drifts.npy", allow_pickle=True)[:, 1:, 0]
#nad_drifts = np.load(root_dir+extra_path+"_nad_drifts.npy", allow_pickle=True)[:, 1:, 0]
#ridge_drifts = np.load(root_dir+extra_path+"_ridge_drifts.npy", allow_pickle=True)[:, 1:, 0]
#hermite_drifts = np.load(root_dir+extra_path+"_hermite_drifts.npy", allow_pickle=True)[:, 1:, 0]

In [ ]:
# Cloud plot of differences
hermite_diffs = true_paths - hermite_paths
ridge_diffs = true_paths - ridge_paths

In [ ]:
import matplotlib.pyplot as plt
time_ax = np.linspace(1/256, 1/256*(true_paths.shape[1]),true_paths.shape[1])
for t in range(1,true_paths.shape[0], 100):
    plt.plot(time_ax, hermite_paths[t, :], label="Hermite")
    plt.plot(time_ax, ridge_paths[t, :], label="Ridge")
    plt.plot(time_ax, true_paths[t, :], label="True")
    plt.ylabel("Hermite State Difference")
    plt.xlabel("Ridge State Difference")
    plt.title(f"Path")
    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
for t in range(1,hermite_diffs.shape[1], 10):
    plt.scatter(hermite_diffs[:, t], ridge_diffs[:, t], s=1)
    plt.ylabel("Hermite State Difference")
    plt.xlabel("Ridge State Difference")
    plt.title(f"State Difference at time IDX {t+1}")
    plt.show()

In [ ]:
score_eval=pd.read_parquet(root_dir + extra_path+ "_score_MSE.parquet", engine="fastparquet")
nad_eval=pd.read_parquet(root_dir + extra_path+ "_nad_MSE.parquet")
hermite_eval=pd.read_parquet(root_dir + extra_path+ "_hermite_MSE.parquet")
ridge_eval=pd.read_parquet(root_dir + extra_path+ "_ridge_MSE.parquet")

nad_eval_true_law=pd.read_parquet(root_dir + extra_path+"_nad_true_law_MSE.parquet")
score_eval_true_law=pd.read_parquet(root_dir + extra_path+"_score_true_law_MSE.parquet")
hermite_eval_true_law=pd.read_parquet(root_dir + extra_path+"_hermite_true_law_MSE.parquet")
ridge_eval_true_law=pd.read_parquet(root_dir + extra_path+"_ridge_true_law_MSE.parquet")

score_state_eval=pd.read_parquet(root_dir + extra_path+ "_score_state_MSE.parquet")
nad_state_eval=pd.read_parquet(root_dir + extra_path+ "_nad_state_MSE.parquet")
hermite_state_eval=pd.read_parquet(root_dir + extra_path+ "_hermite_state_MSE.parquet")
ridge_state_eval=pd.read_parquet(root_dir + extra_path+ "_ridge_state_MSE.parquet")


nad_unif =pd.read_parquet(root_dir + extra_path+ "_nad_uniform_MSE.parquet", engine="fastparquet")
score_unif =pd.read_parquet(root_dir + extra_path+ "_score_uniform_MSE.parquet", engine="fastparquet")
ridge_unif =pd.read_parquet(root_dir + extra_path+ "_ridge_uniform_MSE.parquet", engine="fastparquet")
hermite_unif =pd.read_parquet(root_dir + extra_path+ "_hermite_uniform_MSE.parquet", engine="fastparquet")



score_eval_std=pd.read_parquet(root_dir + extra_path+ "_score_MSE_STD.parquet", engine="fastparquet")
nad_eval_std=pd.read_parquet(root_dir + extra_path+ "_nad_MSE_STD.parquet")
hermite_eval_std=pd.read_parquet(root_dir + extra_path+ "_hermite_MSE_STD.parquet")
ridge_eval_std=pd.read_parquet(root_dir + extra_path+ "_ridge_MSE_STD.parquet")

nad_eval_true_law_std=pd.read_parquet(root_dir + extra_path+"_nad_true_law_MSE_STD.parquet")
score_eval_true_law_std=pd.read_parquet(root_dir + extra_path+"_score_true_law_MSE_STD.parquet")
hermite_eval_true_law_std=pd.read_parquet(root_dir + extra_path+"_hermite_true_law_MSE_STD.parquet")
ridge_eval_true_law_std=pd.read_parquet(root_dir + extra_path+"_ridge_true_law_MSE_STD.parquet")

score_state_eval_std=pd.read_parquet(root_dir + extra_path+ "_score_state_MSE_STD.parquet")
nad_state_eval_std=pd.read_parquet(root_dir + extra_path+ "_nad_state_MSE_STD.parquet")
hermite_state_eval_std=pd.read_parquet(root_dir + extra_path+ "_hermite_state_MSE_STD.parquet")
ridge_state_eval_std=pd.read_parquet(root_dir + extra_path+ "_ridge_state_MSE_STD.parquet")



quadsin_state_eval = (pd.concat([score_state_eval.rename({d:d+"Score State" for d in score_state_eval.columns}, axis=1),nad_state_eval.rename({d:d+"Nadaraya State" for d in nad_state_eval.columns}, axis=1), ridge_state_eval.rename({d:d+"Ridge State" for d in ridge_state_eval.columns}, axis=1), hermite_state_eval.rename({d:d+"Hermite State" for d in hermite_state_eval.columns}, axis=1)], axis=1)).iloc[:257,:]

quadsin_state_eval_std = (pd.concat([score_state_eval_std.rename({d:d+"Score State" for d in score_state_eval_std.columns}, axis=1),nad_state_eval_std.rename({d:d+"Nadaraya State" for d in nad_state_eval_std.columns}, axis=1), ridge_state_eval_std.rename({d:d+"Ridge State" for d in nad_state_eval_std.columns}, axis=1), hermite_state_eval_std.rename({d:d+"Hermite State" for d in hermite_state_eval_std.columns}, axis=1)], axis=1)).iloc[:257,:]

quadsin_true_law_eval = (pd.concat([score_eval_true_law.rename({d:d+"Score True" for d in score_eval_true_law.columns}, axis=1),nad_eval_true_law.rename({d:d+"Nadaraya True" for d in nad_eval_true_law.columns}, axis=1), ridge_eval_true_law.rename({d:d+"Ridge True" for d in ridge_eval_true_law.columns}, axis=1), hermite_eval_true_law.rename({d:d+"Hermite True" for d in hermite_eval_true_law.columns}, axis=1)], axis=1)).iloc[:256,:]

quadsin_true_law_eval_std = (pd.concat([score_eval_true_law_std.rename({d:d+"Score True" for d in score_eval_true_law_std.columns}, axis=1),nad_eval_true_law_std.rename({d:d+"Nadaraya True" for d in nad_eval_true_law_std.columns}, axis=1), ridge_eval_true_law_std.rename({d:d+"Ridge True" for d in ridge_eval_true_law_std.columns}, axis=1), hermite_eval_true_law_std.rename({d:d+"Hermite True" for d in hermite_eval_true_law_std.columns}, axis=1)], axis=1)).iloc[:256,:]


quadsin_alt_law_eval = (pd.concat([score_eval.rename({d:d+"Score" for d in score_eval.columns}, axis=1),nad_eval.rename({d:d+"Nadaraya" for d in nad_eval.columns}, axis=1), ridge_eval.rename({d:d+"Ridge" for d in ridge_eval.columns}, axis=1), hermite_eval.rename({d:d+"Hermite" for d in hermite_eval.columns}, axis=1)], axis=1)).iloc[:256,:]

quadsin_alt_law_eval_std = (pd.concat([score_eval_std.rename({d:d+"Score" for d in score_eval_std.columns}, axis=1),nad_eval_std.rename({d:d+"Nadaraya" for d in nad_eval_std.columns}, axis=1), ridge_eval_std.rename({d:d+"Ridge" for d in ridge_eval_std.columns}, axis=1), hermite_eval_std.rename({d:d+"Hermite" for d in hermite_eval_std.columns}, axis=1)], axis=1)).iloc[:256,:]

quadsin_uniform = (pd.concat([score_unif.rename({d:d+"Score" for d in score_unif.columns}, axis=1),nad_unif.rename({d:d+"Nadaraya" for d in nad_unif.columns}, axis=1),ridge_unif.rename({d:d+"Ridge" for d in ridge_unif.columns}, axis=1),hermite_unif.rename({d:d+"Hermite" for d in hermite_unif.columns}, axis=1)], axis=1))
quadsin_uniform.columns = quadsin_uniform.columns.map(lambda x: x.replace("mse", ""))

In [ ]:
true_law_eval = quadsin_true_law_eval.sort_index(axis=1).set_index(time_ax)
true_law_eval_std = quadsin_true_law_eval_std.sort_index(axis=1).set_index(time_ax)

In [ ]:
alt_law_eval = quadsin_alt_law_eval.sort_index(axis=1).set_index(time_ax)
alt_law_eval_std = quadsin_alt_law_eval.sort_index(axis=1).set_index(time_ax)

In [ ]:
state_eval = quadsin_state_eval.iloc[1:, :].sort_index(axis=1).set_index(time_ax)
state_eval_std = quadsin_state_eval_std.iloc[1:, :].sort_index(axis=1).set_index(time_ax)

In [ ]:
uniform = quadsin_uniform

In [ ]:
import matplotlib.pyplot as plt
for k in range(0, true_law_eval.shape[1], 4):
    fig, ax = plt.subplots(figsize=(16, 10))
    currevals = true_law_eval.iloc[:, k:k+4]
    currevals=currevals.rename({d:d.replace("True", "").replace("Nadaraya", " Nadaraya").replace("Ridge", " Ridge").replace("Hermite", " Hermite").replace("Score", " Score") for d in currevals.columns}, axis=1)
    label = currevals.columns[0].split(" Hermite")[0]
    if label in "BiPot":
        titlab = "$\mu_3$"
    elif label in "QuadSinHF":
        titlab = "$\mu_2$"
    else:
        titlab = "$\mu_1$"
    currevals = currevals.rename({d: d.split(f"{label}")[-1]for d in currevals.columns}, axis=1)
    currevals.plot(ax=ax, linewidth=4)
    plt.legend(fontsize=34)
    ax.tick_params(labelsize=36)
    plt.title(rf"$E^{{(\mu)}}_j$ for {titlab}", fontsize=50)
    plt.ylabel(rf"$E^{{(\mu)}}_j$", fontsize=45)
    plt.xlabel("Time Axis ($t_j$)", fontsize=45)
    plt.grid(True)
    plt.yscale("log")
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
import matplotlib.pyplot as plt
for k in range(0, alt_law_eval.shape[1], 4):
    fig, ax = plt.subplots(figsize=(16, 10))
    currevals = alt_law_eval.iloc[:, k:k+4]
    currevals=currevals.rename({d:d.replace("True", "").replace("Nadaraya", " Nadaraya").replace("Ridge", " Ridge").replace("Hermite", " Hermite").replace("Score", " Score") for d in currevals.columns}, axis=1)
    label = currevals.columns[0].split(" Hermite")[0]
    if label == "BiPot":
        titlab = "$\mu_3$"
    elif label == "QuadSinHF":
        titlab = "$\mu_2$"
    else:
        titlab = "$\mu_1$"
    currevals = currevals.rename({d: d.split(f" ")[-1]for d in currevals.columns}, axis=1)*1/256
    currevals.plot(ax=ax, linewidth=4)
    plt.legend(fontsize=34)
    ax.tick_params(labelsize=36)
    plt.title(rf"$\widetilde{{E}}^{{(\mu)}}_j$ for {titlab}", fontsize=50)
    plt.ylabel(rf"$\widetilde{{E}}^{{(\mu)}}_j$", fontsize=45)
    plt.xlabel("Time Axis ($t_j$)", fontsize=45)
    plt.grid(True)
    plt.yscale("log")
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
import matplotlib.pyplot as plt
for k in range(0, state_eval.shape[1], 4):
    fig, ax = plt.subplots(figsize=(16, 10))
    currevals = state_eval.iloc[:, k:k+4]
    currevals=currevals.rename({d:d.replace("State", "").replace("Nadaraya", " Nadaraya").replace("Score", " Score").replace("Hermite", " Hermite").replace("Ridge", " Ridge") for d in currevals.columns}, axis=1)
    label = currevals.columns[0].split(" Hermite")[0]
    if label == "BiPot":
        titlab = "$\mu_3$"
    elif label == "QuadSinHF":
        titlab = "$\mu_2$"
    else:
        titlab = "$\mu_1$"
    currevals = currevals.rename({d: d.split(f"{label}")[-1]for d in currevals.columns}, axis=1)
    currevals.plot(ax=ax, linewidth=4)
    plt.legend(fontsize=34)
    ax.tick_params(labelsize=36)
    plt.title(rf"$E^{{(Y)}}_j$ for {titlab}", fontsize=50)
    plt.ylabel(rf"$E^{{(Y)}}_j$", fontsize=45)
    plt.xlabel("Time Axis ($t_j$)", fontsize=45)
    plt.grid(True)
    plt.yscale("log")
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
valsign = {"SinLog":1, "QuadSinHF":2, "BiPot":3}
def mapping(y):
    if "Nadaraya" in y:
        return "Nadaraya"
    elif "Score" in y:
        return "Score"
    elif "Hermite" in y:
        return "Hermite"
    return "Ridge"

In [ ]:
s=(true_law_eval.iloc[[-1],:].T.assign(M =lambda x: x.index.map(lambda y: y.replace("Nadaraya", "").replace("Hermite", "").replace("Ridge", "").replace("Score", "").replace(" True", ""))).assign(D= lambda x: x.index.map(lambda y: mapping(y))).assign(V=lambda x: x.M.map(lambda y: valsign[y])).sort_values("V").set_index("D").groupby(level=0, axis=0, group_keys=True).apply(lambda x: x.set_index("M")).iloc[:, 0].unstack(level=-1).iloc[::-1, :]*100).round(3)
s

In [ ]:
s=(alt_law_eval.iloc[[-1],:].T.assign(M =lambda x: x.index.map(lambda y: y.replace("Nadaraya", "").replace("Hermite", "").replace("Ridge", "").replace("Score", "").replace(" True", ""))).assign(D= lambda x: x.index.map(lambda y: mapping(y))).assign(V=lambda x: x.M.map(lambda y: valsign[y])).sort_values("V").set_index("D").groupby(level=0, axis=0, group_keys=True).apply(lambda x: x.set_index("M")).iloc[:, 0].unstack(level=-1).iloc[::-1, :]*100).round(3)
s

In [ ]:
s=state_eval.iloc[[-1],:].T.assign(M =lambda x: x.index.map(lambda y: y.replace("Nadaraya", "").replace("Hermite", "").replace("Ridge", "").replace("Score", "").replace(" State", ""))).assign(D= lambda x: x.index.map(lambda y: mapping(y))).assign(V=lambda x: x.M.map(lambda y: valsign[y])).sort_values("V").set_index("D").groupby(level=0, axis=0, group_keys=True).apply(lambda x: x.set_index("M")).iloc[:, 0].unstack(level=-1).iloc[::-1, :]*1000
s.round(3)

In [ ]:
uniform